In [ ]:
import pandas as pd

def transmission_to_dataframe(transmission_saved, address_saved):

    if transmission_saved is None:
        return None

    ip_address = address_saved[0]

    transmission = transmission_saved.decode('UTF-8')

    # Clean transmission:
    transmission = transmission.split(',')
    transmission = [x.lstrip(' ') for x in transmission]

    # Confirm start and end of transmission
    check1 = transmission[0] == 'SOT'
    check2 = transmission[-1] == 'EOT'
    transmission_status = True if check1 and check2 else False

    # Remove SOT and EOT
    transmission = transmission[1:-1]

    # Convert to dictionary
    keys = [x.split(' ')[0] for x in transmission]
    values = [x.split(' ')[1] for x in transmission]
    weather_data = {}
    weather_data['ip_address'] = ip_address
    weather_data['Timestamp'] = time.time()
    weather_data['transmission_status'] = transmission_status

    for key, value in zip(keys, values):
        divisor = 1
        for factor in [10, 100, 1000, 10000]:
            if '_%s_' % factor in key:
                divisor = factor
                key = key.replace('_%s' % factor, '')
        weather_data[key] = [float(value) / divisor]

    df_data = pd.DataFrame.from_dict(weather_data)
    return df_data


In [12]:
# https://realpython.com/python-sockets/#tcp-sockets
# https://techtutorialsx.com/2018/05/17/esp32-arduino-sending-data-with-socket-client/

import socket
import time
import os

HOST = "192.168.4.102"  # Standard loopback interface address (localhost)
PORT = 8090  # Port to listen on (non-privileged ports are > 1023)

collection_start_time = time.time()
collection_end_time = 60 * 60 * 24
collection_end_time = 120
filename = 'Weather_Stick_Data.csv'
timeout = 5


filepath = os.path.join('./', filename)

df_data = None

while True:

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        # Time out options
        
        transmission_saved = None

        conn, address = s.accept()
        with conn:

            connection_start_time = time.time()

            print(f"Connected by {address}")
            while True:

                try:
                    conn.settimeout(5)

                    connection_time = time.time() - connection_start_time

                    transmission = conn.recv(1024)
                    if not transmission:
                        break
                    print(transmission)
                    transmission_saved = transmission
                    address_saved = address

                    # Break from connection if it's taking too long
                    if connection_time > timeout:
                        transmission_saved = None
                        break
                except:
                    transmission_saved = None
                    break

    df_transmission = transmission_to_dataframe(transmission_saved, address_saved)
    if df_data is None and df_transmission is not None:
        df_data = pd.read_csv(filepath)
        df_data = pd.concat([df_data, df_transmission]) 
        df_data.to_csv(filepath, index=False)
    elif df_transmission is not None:
        df_data = pd.concat([df_data, df_transmission]) 
        df_data.to_csv(filepath, index=False)

    if time.time() - collection_start_time > collection_end_time:
        break

    print(" ")

Connected by ('192.168.6.172', 59705)
b'SOT, scd_co2_ppm 762, scd_temperature_10_degc 232, scd_humidity_10_percent 511, tsl_light_fs 3447, tsl_light_ir 710, tsl_light_vis 2737, bme_temperature_100_degc 2702, bme_pressure_pa 101059, bme_humidity_1000_percent 40933, bme_gas 53274, bme_iaq 0, max_level_10_percent 70, max_level_10_percent 70, max_batt_10_voltage 38, max_avg_current_ua 1733125, max_temperature_100_degc 2656, EOT'
 
Connected by ('192.168.6.172', 59706)
b'SOT, scd_co2_ppm 777, scd_temperature_10_degc 239, scd_humidity_10_percent 504, tsl_light_fs 3451, tsl_light_ir 710, tsl_light_vis 2741, bme_temperature_100_degc 2751, bme_pressure_pa 101057, bme_humidity_1000_percent 39657, bme_gas 9737, bme_iaq 340, max_level_10_percent 80, max_level_10_percent 80, max_batt_10_voltage 39, max_avg_current_ua 1716093, max_temperature_100_degc 2700, EOT'
 
Connected by ('192.168.4.172', 59707)
b'SOT, scd_co2_ppm 768, scd_temperature_10_degc 245, scd_humidity_10_percent 498, tsl_light_fs 3457

In [13]:
df_data

,ip_address,Timestamp,scd_co2_ppm,scd_temperature_degc,scd_humidity_percent,tsl_light_fs,tsl_light_ir,tsl_light_vis,bme_temperature_degc,bme_pressure_pa,bme_humidity_percent,bme_gas,bme_iaq,max_level_percent,max_batt_voltage,max_avg_current_ua,max_temperature_degc,transmission_status
0,192.168.6.172,9.780000e+02,21.6,57.7,0.0,0.0,0.0,157.91,99411.00,61.038,7689.000,400.0,100.0,4.1,29531.0,24.37,NaN,NaN
1,192.168.4.125,1.684819e+09,657.0,26.0,43.5,3179.0,646.0,2533.00,30.92,101123.000,33.013,2231.0,400.0,99.0,4.1,533750.00,28.43,1.0
2,192.168.4.125,1.684820e+09,669.0,26.0,43.4,3179.0,646.0,2533.00,30.96,101121.000,32.927,2276.0,400.0,99.0,4.1,526562.00,28.37,1.0
3,192.168.6.172,1.684820e+09,696.0,22.5,51.8,776.0,178.0,598.00,26.73,101069.000,41.101,5208711.0,0.0,4.4,3.6,-80625.00,26.00,1.0
4,192.168.4.125,1.684820e+09,660.0,26.1,43.3,3183.0,647.0,2536.00,31.00,101123.000,32.847,2319.0,400.0,99.0,4.1,522656.00,28.50,1.0
5,192.168.4.125,1.684820e+09,652.0,26.1,43.0,3185.0,647.0,2538.00,31.05,101119.000,32.739,3367.0,400.0,99.0,4.1,517500.00,28.50,1.0
6,192.168.4.125,1.684820e+09,641.0,26.3,42.4,3190.0,648.0,2542.00,31.32,101119.000,32.075,3741.0,400.0,99.0,4.1,483281.00,28.56,1.0
7,192.168.4.125,1.684820e+09,663.0,26.3,42.2,3187.0,647.0,2540.00,31.33,101117.000,32.053,3783.0,400.0,99.0,4.1,477343.00,28.62,1.0
8,192.168.4.125,1.684820e+09,652.0,26.3,42.1,3189.0,648.0,2541.00,31.35,101117.000,32.019,3847.0,400.0,99.0,4.1,473437.00,28.68,1.0
9,192.168.6.172,1.684820e+09,743.0,22.5,51.8,3438.0,708.0,2730.00,26.78,101069.000,41.073,4214391.0,0.0,4.9,3.8,1729531.00,26.06,1.0


In [ ]:
# 192.168.4.174 - Device 2 in office
# 192.168.4.172 - Device 1 in bedroom
# 192.168.4.125 - Device 3

In [5]:
# df_data = pd.read_csv('./second_data.csv')
import pandas as pd
df_data = pd.read_csv('./Weather_Stick_Data.csv')
df_data

,ip_address,scd_co2_ppm,scd_temperature_degc,scd_humidity_percent,tsl_light_fs,tsl_light_ir,tsl_light_vis,bme_temperature_degc,bme_pressure_pa,bme_humidity_percent,bme_gas,bme_iaq,max_level_percent,max_batt_voltage,max_avg_current_ua,max_temperature_degc,transmission_status
0,192.168.6.172,978.0,21.6,57.7,0.0,0.0,0.0,157.91,99411.0,61.038,7689.0,400.0,100.0,4.1,29531.0,24.37,NaN
1,1684819143.8167598,654.0,24.3,48.5,3145.0,641.0,2504.0,28.15,101125.0,38.185,1114.0,375.0,99.0,4.1,616718.0,27.18,True
2,1684819165.905939,631.0,24.5,47.9,3160.0,643.0,2517.0,28.56,101123.0,37.415,1108.0,385.0,99.0,4.1,610781.0,27.37,True
